In [ ]:
# Databricks notebook source
import matplotlib.pyplot as plt
import networkx as nx
from utils import format_data
import os
import pandas as pd
import numpy as np

COMMAND ----------

In [ ]:
from databricks_access_layer.db_connector import DBConnector
from databricks_access_layer.query import Query
from databricks_access_layer.credentials import SnowflakeCredentials, SecretsManagerProvider

In [ ]:
snowflake_credentials = SecretsManagerProvider(secret_name='airflow/connections/dal').get_credentials()
snow_runner = DBConnector(credentials=snowflake_credentials)

In [ ]:
conversations_public = Query('queries/conversations_public.sql')
dict_cursor = snow_runner.get_data_dict(conversations_public)

COMMAND ----------

In [ ]:
df = pd.DataFrame(dict_cursor)
df['projects_per_user'] = df.groupby(['USER_ID'])['PROJECT_ID'].transform('nunique')
df['users_per_project'] = df.groupby(['PROJECT_ID'])['USER_ID'].transform('nunique')
df['projects_per_user_quantile'] = pd.qcut(df['projects_per_user'], 10, labels=False)
df['users_per_project_quantile'] = pd.qcut(df['users_per_project'], 3, labels=False)

COMMAND ----------

In [ ]:
user_roles = Query('queries/user_roles.sql')
dict_cursor_roles = snow_runner.get_data_dict(user_roles)

COMMAND ----------

In [ ]:
def filtered_users(df:pd.DataFrame, 
                   quantile:int, 
                   data:list) -> list: 
    user_list = list(set(df[df.users_per_project_quantile == quantile].USER_ID))
    return list(filter(lambda x: x['USER_ID'] in user_list, data))

In [ ]:
def build_network(data:list, roles:list) -> nx.Graph:
    interactions_list, users_list = format_data(data)
    interactions_list = [i +  (len(list(filter(lambda interaction: interaction == i, interactions_list))),) for i in interactions_list]
    G = nx.Graph()
    G.add_nodes_from(users_list)
    G.add_weighted_edges_from(interactions_list)
    attrs = {i['USER_ID']:{"POSITION": i['POSITION'], "ROLE": i['ROLE'], "JOB_CATEGORY": i['JOB_CATEGORY']}  for i in roles}
    nx.set_node_attributes(G, attrs)
    return G

In [ ]:
def centrality(G:nx.Graph, eigen_vector:bool = True) -> dict:
    if eigen_vector:
        centrality = nx.eigenvector_centrality(G, max_iter=1000, tol=1e-06, nstart=None, weight=None) 
    else:
        centrality = nx.pagerank(G, alpha=0.85)
    return centrality

COMMAND ----------

In [ ]:
projects = list(set(df.PROJECT_ID))
list_net = []
for project in  projects:
    user_list = list(set(df[df.PROJECT_ID == project].USER_ID))
    data = list(filter(lambda x: x['USER_ID'] in user_list, dict_cursor))
    roles = list(filter(lambda x: x['USER_ID'] in user_list, dict_cursor_roles))
    list_net.append(build_network(data, roles))

COMMAND ----------

In [ ]:
centrality_dict = dict()
for i in range(3):
    data = filtered_users(df, i, dict_cursor)
    G = build_network(data, dict_cursor_roles)
    creatives = [x[0] for x in list(filter(lambda x: x[1] == 'Creative', list(G.nodes(data = "ROLE"))))]
    H = G.subgraph(creatives)
    centrality_dict = centrality_dict | centrality(H, eigen_vector=True)

COMMAND ----------

In [ ]:
centrality_dict = dict(sorted(centrality_dict.items(), key=lambda item: item[1], reverse=True ))

COMMAND ----------

In [ ]:
creative_mrr = Query('queries/creative_mrr.sql')
creative_mrr_cursor = snow_runner.get_data_dict(creative_mrr)

COMMAND ----------

In [ ]:
df_creative_mrr = pd.DataFrame(creative_mrr_cursor)

COMMAND ----------

In [ ]:
df_creative_mrr['cetrality'] = df_creative_mrr.USER_ID.map(centrality_dict)

COMMAND ----------

In [ ]:
df_creative_mrr.AVG_HOURS_LOGGED = df_creative_mrr.AVG_HOURS_LOGGED.apply(lambda x:float(x))

In [ ]:
mean_avg_hours_logged = np.mean(df_creative_mrr.AVG_HOURS_LOGGED)
std_avg_hours_logged = np.std(df_creative_mrr.AVG_HOURS_LOGGED)

In [ ]:
df_creative_mrr.AVG_HOURS_LOGGED = df_creative_mrr.AVG_HOURS_LOGGED.apply(lambda x: (x - mean_avg_hours_logged)/std_avg_hours_logged)

In [ ]:
mean_centrality = np.mean(df_creative_mrr.cetrality)
std_centrality = np.std(df_creative_mrr.cetrality)

In [ ]:
df_creative_mrr.cetrality = df_creative_mrr.cetrality.apply(lambda x: (x - mean_centrality)/std_centrality)

COMMAND ----------

In [ ]:
df_creative_mrr = df_creative_mrr[df_creative_mrr.cetrality.notna()]

COMMAND ----------

In [ ]:
len(df_creative_mrr)

COMMAND ----------

In [ ]:
df_creative_mrr.corr()

COMMAND ----------

In [ ]:
df_creative_mrr[df_creative_mrr.USER_ID.isin(['U4AMYQDAN', 'U026VFR8R3K','U029PBLH52N','U02BMKLR9NJ','U03KTQUJBFE','U02BPFFAXK2'])]

COMMAND ----------

In [ ]:
df[df.USER_ID.isin(['U4AMYQDAN', 'U026VFR8R3K','U029PBLH52N','U02BMKLR9NJ','U03KTQUJBFE','U02BPFFAXK2']) & df.REPLY_USERS.notna()]

COMMAND ----------